In [1]:
import numpy as np
import MDAnalysis as mda
import nmrformd

/home/simon/.local/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [2]:
import sys, os, git
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")

In [3]:
def save_result(data, name = "intra_H2O", aniso = False):
    """Save the correlation functions in dictionary"""
    if not os.path.exists("raw_data/"):
        os.makedirs("raw_data/")
    saving_file = "raw_data/" + name + ".npy"
    t = data.t
    f = data.f
    if aniso:
        C1 = data.gij[0]
        C2 = data.gij[1]
        C3 = data.gij[2]
    else:
        C = data.gij[0]
    R1 = data.R1
    R2 = data.R2
    N = data.group_j.atoms.n_atoms
    try:
        previous_dictionary = np.load(saving_file, allow_pickle=True)
        t_prev = np.real(previous_dictionary.item()["t"])
        assert len(t_prev) == len(t)
        if aniso:
            C1_prev = np.real(previous_dictionary.item()["C1"])
            C2_prev = np.real(previous_dictionary.item()["C2"])
            C3_prev = np.real(previous_dictionary.item()["C3"])
        else:
            C_prev = np.real(previous_dictionary.item()["C"])
        R1_prev = np.real(previous_dictionary.item()["R1"])
        R2_prev = np.real(previous_dictionary.item()["R2"])
        N_prev = np.real(previous_dictionary.item()["N"])
        if aniso:
            C1 = (C1*N + C1_prev*N_prev) / (N_prev + N)
            C2 = (C2*N + C2_prev*N_prev) / (N_prev + N)
            C3 = (C3*N + C3_prev*N_prev) / (N_prev + N)
        else:
            C = (C*N + C_prev*N_prev) / (N_prev + N)
        R1 = (R1*N + R1_prev*N_prev) / (N_prev + N)
        R2 = (R2*N + R2_prev*N_prev) / (N_prev + N)
        N += N_prev
    except:
        pass
    if aniso:
        dictionary = {
        "t": t,
        "f": f,
        "C1": C1,
        "C2": C2,
        "C3": C3,
        "N": N,
        "R1": R1,
        "R2": R2,
        }
    else:
        dictionary = {
        "t": t,
        "f": f,
        "C": C,
        "N": N,
        "R1": R1,
        "R2": R2,
        }
    np.save(saving_file, dictionary)
    return N

In [10]:
while 1<2:
    # vary T
    #for T in [320]:
    # datapath = git_path + "/data/bulk-water-tip4p/bulk-water/raw-data/N4000_" + str(T) + "K/"

    T = 320
    datapath = "/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/"
    xtc = []
    for x in np.arange(1, 11):
        xtc.append(datapath+"prod"+str(x)+".xtc")
    u = mda.Universe(datapath+"prod1.tpr", xtc)
    hydrogen = u.select_atoms("name HW1 HW2")
    n_hydrogen = hydrogen.n_atoms
    intra = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "intra_molecular", number_i=5)
    inter = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "inter_molecular", number_i=1)
    n_intra = save_result(intra, name = "N4000_intra_T" + str(T) + "K-b")
    n_inter = save_result(inter, name = "N4000_inter_T" + str(T) + "K-b")
    print(n_hydrogen, T, n_intra)

8000 320 1
8000 320 2
8000 320 3
8000 320 4
8000 320 5
8000 320 6
8000 320 7
8000 320 8
8000 320 9


KeyboardInterrupt: 

In [9]:
xtc



['/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod1.xtc',
 '/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod2.xtc',
 '/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod3.xtc',
 '/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod4.xtc',
 '/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod5.xtc',
 '/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod6.xtc',
 '/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod7.xtc',
 '/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod8.xtc',
 '/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod9.xtc',
 '/home/simon/Git/NMR/raw-data/bulk-water-tip4p-temp/bulk-water/N4000_320K/prod10.xtc']

In [4]:
while 1<2:
    # varying number
    if 1 > 2:
        mystep = 1
        for N in ["N25", "N39", "N62", "N99", "N158", "N251", "N398", "N631", "N1002", "N1589", "N2521"]:
            datapath = git_path + "/data/bulk-water-tip4p/bulk-water/raw-data/"+N+"/"
            for n in np.arange(1, 10):
                if (os.path.exists(datapath+"prod"+str(n)+".xtc")) & (os.path.exists(datapath+"topology.data")):
                    u = mda.Universe(datapath+"topology.data", datapath+"prod"+str(n)+".xtc")
                    u.transfer_to_memory(step = mystep)
                    hydrogen = u.select_atoms("type 2")
                elif (os.path.exists(datapath+"prod"+str(n)+".xtc")) & (os.path.exists(datapath+"prod1.tpr")):
                    u = mda.Universe(datapath+"prod1.tpr", datapath+"prod"+str(n)+".xtc")
                    u.transfer_to_memory(step = mystep)
                    hydrogen = u.select_atoms("name HW1 HW2")
                dt = np.round(u.trajectory.dt,2)
                n_hydrogen = hydrogen.n_atoms
                intra = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "intra_molecular", number_i=5)
                inter = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "inter_molecular", number_i=1)
                n_intra = save_result(intra, name = N + "_intra_dt" + str(dt))
                n_inter = save_result(inter, name = N + "_inter_dt" + str(dt))
                if n == 1:
                    print(n_hydrogen, mystep, n_intra)
    # varying step
    N = "N4000"
    for mystep in [1, 2, 4, 8, 16, 32]:
        datapath = git_path + "/data/bulk-water-tip4p/bulk-water/raw-data/"+N+"/"
        xtcs = [[datapath+"prod1.xtc", datapath+"prod2.xtc"],
                [datapath+"prod3.xtc", datapath+"prod4.xtc"]]
        for xtc in xtcs:
            u = mda.Universe(datapath+"prod1.tpr", xtc)
            u.transfer_to_memory(step = mystep)
            dt = np.round(u.trajectory.dt,2)
            hydrogen = u.select_atoms("name HW1 HW2")
            n_hydrogen = hydrogen.n_atoms
            intra = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "intra_molecular", number_i=5)
            inter = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "inter_molecular", number_i=1)
            n_intra = save_result(intra, name = N + "_intra_dt" + str(dt))
            n_inter = save_result(inter, name = N + "_inter_dt" + str(dt))
            if mystep == 1:
                print(n_hydrogen, mystep, n_intra, dt)
    # aniso
    N = "N4000"
    datapath = git_path + "/data/bulk-water-tip4p/bulk-water/raw-data/"+N+"/"
    xtcs = [[datapath+"prod1.xtc", datapath+"prod2.xtc"],
            [datapath+"prod3.xtc", datapath+"prod4.xtc"]]
    for xtc in xtcs:
        u = mda.Universe(datapath+"prod1.tpr", xtc)
        dt = np.round(u.trajectory.dt,2)
        hydrogen = u.select_atoms("name HW1 HW2")
        n_hydrogen = hydrogen.n_atoms
        intra = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen,
                             type_analysis = "intra_molecular", number_i=5, isotropic = False)
        inter = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen,
                             type_analysis = "inter_molecular", number_i=1, isotropic = False)
        n_intra = save_result(intra, name = N + "_intra_aniso_dt" + str(dt), aniso = True)
        n_inter = save_result(inter, name = N + "_inter_aniso_dt" + str(dt), aniso = True)
        print("N:", n_intra)
    # vary T
    for T in [280, 290, 310, 320]:
        datapath = git_path + "/data/bulk-water-tip4p/bulk-water/raw-data/N4000_" + str(T) + "K/"
        xtc = [datapath+"prod1.xtc", datapath+"prod2.xtc"]
        u = mda.Universe(datapath+"prod1.tpr", xtc)
        hydrogen = u.select_atoms("name HW1 HW2")
        n_hydrogen = hydrogen.n_atoms
        intra = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "intra_molecular", number_i=5)
        inter = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "inter_molecular", number_i=1)
        n_intra = save_result(intra, name = "N4000_intra_T" + str(T) + "K")
        n_inter = save_result(inter, name = "N4000_inter_T" + str(T) + "K")
        print(n_hydrogen, T, n_intra)
    # vary co
    for co in [6, 8, 10, 12]:
        datapath = git_path + "/data/bulk-water-tip4p/bulk-water/raw-data/N4000_co" + str(co) + "/"
        xtc = [datapath+"prod1.xtc", datapath+"prod2.xtc"]
        u = mda.Universe(datapath+"prod1.tpr", xtc)
        hydrogen = u.select_atoms("name HW1 HW2")
        n_hydrogen = hydrogen.n_atoms
        intra = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "intra_molecular", number_i=5)
        inter = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "inter_molecular", number_i=1)
        n_intra = save_result(intra, name = "N4000_intra_co" + str(co))
        n_inter = save_result(inter, name = "N4000_inter_co" + str(co))
        print(n_hydrogen, co, n_intra)



8000 1 555 1.0
8000 1 556 1.0
N: 316
N: 319
8000 280 190
8000 290 190
8000 310 190
8000 320 188
8000 6 179
8000 8 185
8000 10 184
8000 12 183
8000 1 561 1.0
8000 1 562 1.0
N: 322
N: 325
8000 280 193
8000 290 193
8000 310 193
8000 320 191
8000 6 182
8000 8 188
8000 10 187
8000 12 186
8000 1 567 1.0
8000 1 568 1.0
N: 328
N: 331
8000 280 196
8000 290 196
8000 310 196
8000 320 194
8000 6 185
8000 8 191
8000 10 190
8000 12 189
8000 1 573 1.0
8000 1 574 1.0
N: 334
N: 337
8000 280 199
8000 290 199
8000 310 199
8000 320 197
8000 6 188
8000 8 194
8000 10 193
8000 12 192
8000 1 579 1.0
8000 1 580 1.0
N: 340
N: 343
8000 280 202
8000 290 202
8000 310 202
8000 320 200
8000 6 191
8000 8 197
8000 10 196
8000 12 195
8000 1 585 1.0
8000 1 586 1.0
N: 347
N: 349
8000 280 205
8000 290 205
8000 310 205
8000 320 203
8000 6 194
8000 8 200
8000 10 199
8000 12 198
8000 1 591 1.0
8000 1 592 1.0
N: 353
N: 355
8000 280 208
8000 290 208
8000 310 208
8000 320 206
8000 6 197
8000 8 203
8000 10 202
8000 12 201
8000 1